In [1]:
import pandas as pd

# Load Parquet file
df = pd.read_parquet('data.parquet', engine='pyarrow')

# Display the first few rows
print(df.head())


          https://www.domain.com.au/27-ethio-street-tarneit-vic-3029-17191712  \
name                       27 Ethio Street, Tarneit VIC 3029                    
cost_text                                      $550 per week                    
rooms                                         4 Beds,2 Baths                    
parking                                            2 Parking                    
desc                                          Angus Brayshaw                    

          https://www.domain.com.au/14-viking-street-tarneit-vic-3029-17191908  \
name                      14 Viking Street, Tarneit VIC 3029                     
cost_text                                        $550 weekly                     
rooms                                         4 Beds,2 Baths                     
parking                                            1 Parking                     
desc                                              Kayla Crea                     

          https://ww

In [ ]:
#pip3 install openrouteservice
#pip3 install osmnx geopy

In [5]:
import openrouteservice

# Initialize the OpenRouteService client with your API key
client = openrouteservice.Client(key='5b3ce3597851110001cf6248053ae8c7c7c845a6a46874c7f36c4156')  # Replace with your API key

from geopy.geocoders import Nominatim
import time

# Initialize the geocoder
geolocator = Nominatim(user_agent="property_geocoder")

# Example address
address = "27 Ethio Street, Tarneit, VIC, 3029, Australia"

# Geocode the address
location = geolocator.geocode(address)

# Get the coordinates (latitude and longitude)
if location:
    print(f"Coordinates for {address}: ({location.latitude}, {location.longitude})")
else:
    print(f"Could not geocode the address: {address}")



Coordinates for 27 Ethio Street, Tarneit, VIC, 3029, Australia: (-37.8132189, 144.6721558)


In [18]:
import osmnx as ox
from geopy.distance import geodesic
from shapely.geometry import Point, Polygon, MultiPolygon

# Example property coordinates
property_coords = (-37.8132189, 144.6721558)  # (latitude, longitude)

# Set up a query for schools in a larger radius around the property
radius = 2000  # meters (20 km)
tags = {'amenity': 'school'}

try:
    # Use OSMnx to get schools within the radius
    schools = ox.geometries.geometries_from_point(property_coords, tags=tags, dist=radius)

    # Check if schools are found
    if schools.empty:
        print("No schools found within the radius.")
    else:
        school_coords = []
        print(f"Found {len(schools)} schools. Processing...")
        
        # Extract school coordinates, handling Polygon geometries
        for idx, row in schools.iterrows():
            geometry = row.geometry
            if isinstance(geometry, Point):
                # If it's a point, use the point's coordinates
                school_coords.append((geometry.y, geometry.x))
                print(f"School {idx}: Point coordinates: ({geometry.y}, {geometry.x})")
            elif isinstance(geometry, (Polygon, MultiPolygon)):
                # If it's a Polygon or MultiPolygon, use the centroid
                centroid = geometry.centroid
                school_coords.append((centroid.y, centroid.x))
                print(f"School {idx}: Polygon centroid coordinates: ({centroid.y}, {centroid.x})")
            else:
                print(f"School {idx}: Unexpected geometry type: {type(geometry)}")

        if not school_coords:
            print("No valid school coordinates found.")
        else:
            # Function to calculate the distance between property and each school
            def get_distance(school_coord):
                return geodesic(property_coords, school_coord).meters

            # Find the closest school
            closest_school = min(school_coords, key=get_distance)
            distance_to_closest = get_distance(closest_school)

            # Output the result
            print(f"\nClosest school coordinates: {closest_school}")
            print(f"Distance to closest school: {distance_to_closest:.2f} meters")

except ox._errors.InsufficientResponseError:
    print("Error: No data elements in server response. Try increasing radius or check the location.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Found 25 schools. Processing...
School ('way', 31249641): Polygon centroid coordinates: (-37.84761948504883, 144.700787873062)
School ('way', 69006900): Polygon centroid coordinates: (-37.849990901183254, 144.7219196387751)
School ('way', 91356779): Polygon centroid coordinates: (-37.85908320825177, 144.69133621330258)
School ('way', 91357169): Polygon centroid coordinates: (-37.842330381772086, 144.69422781648694)
School ('way', 140349782): Polygon centroid coordinates: (-37.83275036074753, 144.66492369447326)
School ('way', 165023704): Polygon centroid coordinates: (-37.85396333303886, 144.7219433487764)
School ('way', 183937557): Polygon centroid coordinates: (-37.8567655814588, 144.6884116086855)
School ('way', 183979260): Polygon centroid coordinates: (-37.833972078009715, 144.68028117555497)
School ('way', 183979261): Polygon centroid coordinates: (-37.836127851364374, 144.6797301420997)
School ('way', 326481128): Polygon centroid coordinates: (-37.838719817233354, 144.7179640957

/tmp/ipykernel_6425/1714809836.py:14: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  schools = ox.geometries.geometries_from_point(property_coords, tags=tags, dist=radius)
